In [ ]:
import os
import pandas as pd
import zipfile

In [ ]:
# Seteo la ruta del repositorio (reemplazarla por la propia)
ruta = r"C:\Users\Fran\MAESTRÍA\Machine Learning\Tarea 2"

In [ ]:
# Fijo rutas para cada carpeta del repositorio
document_path = os.path.join(ruta, "document")
scripts_path = os.path.join(ruta, "scripts")
stores_path = os.path.join(ruta, "stores")
views_path = os.path.join(ruta, "views")

zip_path = os.path.join(stores_path, "mlunlp-2024-ps-2.zip")


In [ ]:
train_hogares_path = "train_hogares.csv"
train_personas_path = "train_personas.csv"
test_hogares_path = "test_hogares.csv"
test_personas_path = "test_personas.csv"

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(train_hogares_path) as f:
        train_hogares = pd.read_csv(f)

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(train_personas_path) as f:
        train_personas = pd.read_csv(f)

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(test_hogares_path) as f:
        test_hogares = pd.read_csv(f)

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(test_personas_path) as f:
        test_personas = pd.read_csv(f)

In [ ]:
train_merged = pd.merge(train_personas, train_hogares, on='id', how= "inner", suffixes=('_ind', '_hogar'))  
test_merged = pd.merge(test_personas, test_hogares, on='id', how= "inner", suffixes=('_ind', '_hogar'))  

**Creación y transformación de variables**

Cuestiones a tener en cuenta: 

Se cambió el formato de las variables "P6210", "P6100", "P6585s3", "P6585s1" de float a entero.
Hacerlo también para las bases de testeo

In [ ]:
columnas = train_merged.columns.tolist()
print(columnas)

columnas2 = test_merged.columns.tolist()
print(columnas2)

In [ ]:
# Creo la variable hacin
train_merged["hacin"] = train_merged["Nper"] / train_merged["P5010"]
test_merged["hacin"] = test_merged["Nper"] / test_merged["P5010"]

In [ ]:
# Transformo los valores de P6210
train_merged["P6210"] = train_merged["P6210"].fillna(0).astype(int) 
print(train_merged["P6210"].unique())

test_merged["P6210"] = train_merged["P6210"].fillna(0).astype(int) 

In [ ]:
# Obtengo el máximo nivel educativo por hogar
educacion_max_hogar = train_merged.groupby("id")["P6210"].max().reset_index()
educacion_max_hogar2 = test_merged.groupby("id")["P6210"].max().reset_index()

# Renombro la columna
educacion_max_hogar.rename(columns={"P6210": "educ_max"}, inplace=True)
educacion_max_hogar2.rename(columns={"P6210": "educ_max"}, inplace=True)

train_merged = train_merged.merge(educacion_max_hogar, on="id", how="left")
test_merged = test_merged.merge(educacion_max_hogar2, on="id", how="left")

In [ ]:
# Transformo los valores de P6100
train_merged["P6100"] = train_merged["P6100"].fillna(0).astype(int) 
test_merged["P6100"] = test_merged["P6100"].fillna(0).astype(int) 

# Creo dummy con personas de régimen de salud subsidiado
test_merged["salud_subsi"] = (test_merged["P6100"] == 3).astype(int)
train_merged["salud_subsi"] = (train_merged["P6100"] == 3).astype(int)

In [ ]:
# Transformo los valores de P6585s3
train_merged["P6585s3"] = train_merged["P6585s3"].fillna(0).astype(int) 
test_merged["P6585s3"] = test_merged["P6585s3"].fillna(0).astype(int) 

# Asigno valor 1 a quienes reportan que sí y 0 en otro caso
train_merged["fam_subsi"] = (train_merged["P6585s3"] == 1).astype(int)

test_merged["fam_subsi"] = (test_merged["P6585s3"] == 1).astype(int)

In [ ]:
# Transformo los valores de P6585s1
train_merged["P6585s1"] = train_merged["P6585s1"].fillna(0).astype(int) 
test_merged["P6585s1"] = test_merged["P6585s1"].fillna(0).astype(int) 

# Asigno valor 1 a quienes reportan que sí y 0 en otro caso
train_merged["alim_subsi"] = (train_merged["P6585s1"] == 1).astype(int)
test_merged["alim_subsi"] = (test_merged["P6585s1"] == 1).astype(int)

In [ ]:
# Transformo los valores de P6585s2
train_merged["P6585s2"] = train_merged["P6585s2"].fillna(0).astype(int) 
test_merged["P6585s2"] = test_merged["P6585s2"].fillna(0).astype(int) 

# Asigno valor 1 a quienes reportan que sí y 0 en otro caso
train_merged["transp_subsi"] = (train_merged["P6585s2"] == 1).astype(int)
test_merged["transp_subsi"] = (test_merged["P6585s2"] == 1).astype(int)

In [ ]:
# Selecciono las variables que podría llegar a utilizar para estimar pobreza con el primer approach
train_pobreza = train_merged[["id", "Orden", "P6040", "hacin", "Clase_hogar", "educ_max", "salud_subsi", "fam_subsi", "alim_subsi", "transp_subsi", "P6430", "Lp", "Pobre"]]
test_pobreza = test_merged[["id", "Orden", "P6040", "hacin", "Clase_hogar", "educ_max", "salud_subsi", "fam_subsi", "alim_subsi", "transp_subsi", "P6430", "Lp"]]

In [ ]:
# Selecciono las variables que podría llegar a utilizar para estimar pobreza con el segundo approach

train_ingreso = train_merged[["id", "Orden", "Ingtot", "P6020", "P6040", "P6020", "P6920", "P6800", "P6210", "P6426", "P6430","educ_max", "Lp"]]
test_ingreso = test_merged[["id", "Orden", "P6020", "P6040", "P6020", "P6920", "P6800", "P6210", "P6426", "P6430","educ_max", "Lp"]]

# "P6040": edad, 
# "P6020": sexo, 
# "P6920": cotiza pension, 
# "P6800": horas de trabajo semanal normal, 
# "P6210": máximo nivel educativo, 
# "P6426": antigüedad, 
# "P6920": tipo de ocupación